In [1]:
!python --version


Python 3.9.21


In [2]:
2+2

4

In [2]:
from transformers import BertConfig, BertModel
# Building the config
config = BertConfig()

# Building the model from the config
model = BertModel(config)


In [3]:
print(config)

BertConfig {
  "_attn_implementation_autoset": true,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [4]:
# above model is untrained and hence will take time to train again and print gibberish most of the time. 
# So, to spend less time and enerty into it, we'll use pretrained model class 

from transformers import BertModel

# refer to this model card to know more - https://huggingface.co/google-bert/bert-base-cased
model = BertModel.from_pretrained("bert-base-cased")
# output downloaded below are default config parameters and pretrained weights (originally trained by the authors of the model)
# you can change download location via HF_HOME environment variable

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [5]:
# after extra training you can save the models like below 
model.save_pretrained("/home/dvipin2/dev/models/my-bert-base-cased")

In [6]:
"""
models only understand encoded tokens and not sentences, hence first task is to convert string into tokens

for example - my name is khan ====> [[1234,7865,3579,658]] 
and this encoded tokens will be fed to model for training or get the answers.
for more information please refer this link - https://huggingface.co/learn/nlp-course/chapter2/4?fw=pt
"""

'\nmodels only understand encoded tokens and not sentences, hence first task is to convert string into tokens\n\nfor example - my name is khan ====> [[1234,7865,3579,658]] \nand this encoded tokens will be fed to model for training or get the answers.\nfor more information please refer this link - https://huggingface.co/learn/nlp-course/chapter2/4?fw=pt\n'

In [8]:
# Now lets look at tokenizer loading mechanism, below both syntaxes are same
# what they differ is in flexibility

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
tokenizer("Using a Transformer network is simple")


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [2]:
# Autotokenizer

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenizer("Using a Transformer network is simple")
# in the output below bert tokenizer didn't download anything and started form the same checkpoint via AutoTokenizer and output is also same.

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [3]:
#lets save the tokenizer
tokenizer.save_pretrained("/home/dvipin2/dev/models/my-bert-base-cased/")

('/home/dvipin2/dev/models/my-bert-base-cased/tokenizer_config.json',
 '/home/dvipin2/dev/models/my-bert-base-cased/special_tokens_map.json',
 '/home/dvipin2/dev/models/my-bert-base-cased/vocab.txt',
 '/home/dvipin2/dev/models/my-bert-base-cased/added_tokens.json',
 '/home/dvipin2/dev/models/my-bert-base-cased/tokenizer.json')

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/home/dvipin2/dev/models/my-bert-base-cased/")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


In [5]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


In [7]:
decoded_string = tokenizer.decode(ids)
print(decoded_string)

Using a Transformer network is simple


In [11]:
next_seqs = ["Hello!", "Cool.", "Nice!"]
encoded_seq = []
for seq in next_seqs:
    encoded_seq.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(seq)))
print(encoded_seq)



[[8667, 106], [13297, 119], [8835, 106]]


In [12]:
import torch
model_inputs = torch.tensor(encoded_seq)

In [14]:
print(model_inputs)

tensor([[ 8667,   106],
        [13297,   119],
        [ 8835,   106]])


In [16]:
from transformers import AutoModel
model = AutoModel.from_pretrained("/home/dvipin2/dev/models/my-bert-base-cased")
outputs = model(input_ids=model_inputs)

In [17]:
print(outputs)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.3677,  0.6647, -0.2807,  ..., -0.2823,  0.8272,  0.0218],
         [ 0.3931,  0.7670, -0.4687,  ...,  0.6114,  0.9736, -0.4572]],

        [[ 0.0810, -0.5532,  0.1728,  ..., -0.2150,  0.6107,  0.9050],
         [-0.1549, -0.0249, -0.1992,  ...,  0.9487,  0.6500,  0.6370]],

        [[ 0.0029, -0.1294,  0.6850,  ..., -0.8534,  0.4391, -0.2010],
         [-0.2554,  0.3452,  0.6528,  ...,  0.5811,  0.5147,  0.4606]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.2076,  0.3310,  0.9867,  ...,  0.9941, -0.1577,  0.9857],
        [-0.5561,  0.4024,  0.9950,  ...,  0.9969, -0.4081,  0.9893],
        [-0.2240,  0.2850,  0.9639,  ...,  0.9826, -0.1157,  0.9453]],
       grad_fn=<TanhBackward0>), hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None)


In [ ]:
##now you can apply any modeling you want